# Calculating potential loss of transaction intent leakage

Input: a list of transaction hashes that we think have been leaked 
Output: a file with the potential loss in dollars for each transaction as well as the total potential loss in dollar 

## The methodology: 
1. We have a list of transaction hashes that we think have been leaked 
2. We get the details of this transaction using the Infuria and Etherscan APIs
3. We simulate the result of each transaction if it would have been top of the block using the Tenderly API
4. We calculate the difference in dollars for each transaction
5. We sum the potential loss of each transaction to get the total potential loss in dollars. 


## Usage
1. Have a csv file with one column called ' user_tx' (space is important) having all the transactions hashes that you think were leeked
2. open the config_example.py file and follow the instructions for the configuration
3. run the Jupyter Notebook. The output file is called results.csv - but the Jupyter Notebook is full of interesting information.


In [1]:
import pandas as pd
import numpy as np 
import json
import requests 
from config import *

In [2]:
df_main = pd.read_csv(csv_file_path)
#drop the duplicate transaction in case there is any
df_main = df_main.drop_duplicates(subset=[' user_tx'])
df_main.head()

,block_number,user_tx,fees
0,19375342,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,47408284226876256
1,19375361,0x202fc38a52652a0c49927c1771de43939b47e083ba1c...,70226021788882271
2,19375372,0xee51506e07ace44eaad85041210d025ac46241526d2d...,25816233944667279
3,19375384,0x4589dc3b7be6df22ed3657b3310bfff117329a0a7e68...,252120514831501060
4,19375388,0x153a70478d17e082740c30f9d5d20fbca5d298c34cc4...,272612861208189719


In [3]:
#make a list of transaction hash we need to analyse
tx_hash_list = [x for x in df_main[' user_tx'].to_list() if pd.notnull(x)]
print(f'There are {len(tx_hash_list)} transactions')

There are 130 transactions


### Infuria
With this API, we want to get all the inputs necessary to simulate the transaction again later on. Infuria gives us all of these inputs except fot the timestamp of the transaction, which is why we need to use the Etherscan API later on.

#### Call Infuria for the first transaction

In [5]:
#the api key is stored in the config file
url = f"https://mainnet.infura.io/v3/{infura_api_key}"

#Get the infuria response for the first transaction in the list to create a dataframe
payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionByHash",
  "params": [tx_hash_list[0]],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dct = response.json()['result']
dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
df_infuria = pd.DataFrame(dct, index=[0])
df_infuria.head()

,accessList,blockHash,blockNumber,chainId,from,gas,gasPrice,hash,input,maxFeePerGas,maxPriorityFeePerGas,nonce,r,s,to,transactionIndex,type,v,value,yParity
0,None,0x40f2c6a5cca6816f67bc7a3b5e667bd27f7169ec84da...,0x127a4ee,0x1,0xf299dc09ec306e9ed207cdc1296ac6d0d9c5dc7c,0x2bfd7,0xeadb616ac,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,0x7ff36ab5000000000000000000000000000000000000...,0x13ad304380,0x5e69ec0,0x54,0x2a47d3286b8276d24c68cae1e1db8a05c1725b4759ba...,0x3ee3481e8d33fe94124d4e0e76c66694ae5118383e31...,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x1,0x2,0x0,0x429d069189e0000,0x0


#### Call Infuria for all other transactions - !! This will take a few minutes

In [6]:
# get the infuria response for all the other transactions in the list and append the results to the above dataframe
for tx_hash in tx_hash_list[1:]:
  payload = json.dumps({
    "jsonrpc": "2.0",
    "method": "eth_getTransactionByHash",
    "params": [tx_hash],
    "id": 1
  })
  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  if response.ok:
    dct = response.json()['result']
    dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct, index=[0])
    df_infuria = pd.concat([df_infuria, df_temp])
    
  else: 
    print(f"error code {response.status_code} for transaction {tx_hash}")

df_infuria.head()


,accessList,blockHash,blockNumber,chainId,from,gas,gasPrice,hash,input,maxFeePerGas,maxPriorityFeePerGas,nonce,r,s,to,transactionIndex,type,v,value,yParity
0,None,0x40f2c6a5cca6816f67bc7a3b5e667bd27f7169ec84da...,0x127a4ee,0x1,0xf299dc09ec306e9ed207cdc1296ac6d0d9c5dc7c,0x2bfd7,0xeadb616ac,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,0x7ff36ab5000000000000000000000000000000000000...,0x13ad304380,0x5e69ec0,0x54,0x2a47d3286b8276d24c68cae1e1db8a05c1725b4759ba...,0x3ee3481e8d33fe94124d4e0e76c66694ae5118383e31...,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x1,0x2,0x0,0x429d069189e0000,0x0
0,None,0xbfe2748fb504fd42ee26c5d7db6c3cab5d99d56bbde3...,0x127a501,0x1,0x77314da6f40f71c3a850c89e1a05c438a0acd405,0x34479,0xf4c2310e3,0x202fc38a52652a0c49927c1771de43939b47e083ba1c...,0x3593564c000000000000000000000000000000000000...,0x158887c893,0x9402a0,0x4d,0xac3934714a06fc8c26f120276c842b67eee672f2b521...,0x59683f997587e36875f3d1830e8c2c77c8b7983a05d5...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x1,0x2,0x0,0x0,0x0
0,None,0xbc5e86793e57d80ef647106b54cce1a1f2a5b9ecdecc...,0x127a50c,0x1,0x1f7ea43d283d0ef906ee92ddead883a8f078cbc9,0x41bc6,0xff4f60c10,0xee51506e07ace44eaad85041210d025ac46241526d2d...,0x3593564c000000000000000000000000000000000000...,0x15ad608d4c,0x2017a9b,0x416,0xf085eab90fe023a2f519442d5cd4cb054fc63b4b22eb...,0x71ae9fd16ff569a146f8862f036845a40914ae1c27b4...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x4,0x2,0x0,0x0,0x0
0,None,0x6a845bcc4b05da8db0c5da927a0d9867903195a63624...,0x127a518,0x1,0x4e6b065262e3504f2511ef5b8cadc039630803be,0x2e52c,0x1039a71567,0x4589dc3b7be6df22ed3657b3310bfff117329a0a7e68...,0x3593564c000000000000000000000000000000000000...,0x15d0265dd6,0x1c69447,0x29,0x59b8ba6e68ed7261a58ae3d45a00f10cb5aaa064f8d7...,0x42d467edb9dd15965d31f6b444083a95e769294b4213...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x4,0x2,0x1,0x0,0x1
0,None,0xd2dcda496d5c7e73285f9ad24267ab537edbcd14efc6...,0x127a51c,0x1,0xdd3d41d3817abe28519f4f5c0890e9c0f0cfe69b,0x1e8480,0x14f46b0400,0x153a70478d17e082740c30f9d5d20fbca5d298c34cc4...,0xb6f9de95000000000000000000000000000000000000...,NaN,NaN,0x3f4,0x3be3b5b6a8c858a4546cdda89f350555c868e31a6f50...,0x5111bccd8ee8b1dd5dd68bee7d60e3a408884df5fd4f...,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x1,0x0,0x25,0x429d069189e0000,NaN


#### merge infuria response to main

In [7]:
df_main = df_main.merge(df_infuria, left_on = ' user_tx', right_on = 'hash', how ='outer')
df_main.columns

Index(['block_number', ' user_tx', ' fees', 'accessList', 'blockHash',
       'blockNumber', 'chainId', 'from', 'gas', 'gasPrice', 'hash', 'input',
       'maxFeePerGas', 'maxPriorityFeePerGas', 'nonce', 'r', 's', 'to',
       'transactionIndex', 'type', 'v', 'value', 'yParity'],
      dtype='object')

### Etherscan 

We use this API to have the timestamp of the blocks rather than the transactions themselves to reduce the amount of API calls (tx and block time are the same for all tx in the block). We need the timestamp of the transactions because in the Tenderly API, if we do not override the timestamp, then it uses the current time as input variable

In [8]:
#getting all the block numbers of the transactions we want to analyse
block_number_list = list(set([x for x in df_main['block_number'].to_list() if pd.notnull(x)]))
print(f'there are {len(block_number_list)} different blocks')

there are 114 different blocks


#### Get the ehterscan response for the first block in the list to create a dataframe

In [11]:
url_eth = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block_number_list[0]}&apikey={eth_scan_api_key}"

response_eth = requests.request("POST", url_eth)

dct_eth = response_eth.json()['result']
dct_eth = {k: None if not v else v for k, v in dct_eth.items()} # making sure none of the values are empty
df_eth = pd.DataFrame(dct_eth, index=[0])
df_eth.head()

,blockNumber,timeStamp,blockMiner,blockReward,uncles,uncleInclusionReward
0,19376128,1709728067,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,175903410846177465,None,0


#### Get the etherscan responses for the other blocks in the list - !!! This will take a ffew minutes

In [12]:
for block in block_number_list[1:]:
  url_temp = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block}&apikey={eth_scan_api_key}"
  response_temp = requests.request("POST", url_temp)

  if response_temp.ok:
    dct_temp = response_temp.json()['result']
    dct_temp = {k: None if not v else v for k, v in dct_temp.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct_temp, index=[0])
    df_eth = pd.concat([df_eth, df_temp])
    
  else: 
    print(f"error code {response_temp.status_code} for block {block}")

In [13]:
# quick cleaning for later
df_eth['blockNumber'] = df_eth['blockNumber'].astype(int)

#### merge etherscan results to main

In [14]:
df_main = df_main.merge(df_eth, left_on = 'block_number', right_on = 'blockNumber', how ='outer')
df_main.columns

Index(['block_number', ' user_tx', ' fees', 'accessList', 'blockHash',
       'blockNumber_x', 'chainId', 'from', 'gas', 'gasPrice', 'hash', 'input',
       'maxFeePerGas', 'maxPriorityFeePerGas', 'nonce', 'r', 's', 'to',
       'transactionIndex', 'type', 'v', 'value', 'yParity', 'blockNumber_y',
       'timeStamp', 'blockMiner', 'blockReward', 'uncles',
       'uncleInclusionReward'],
      dtype='object')

### Tenderly !! This will take a few minutes
Here we finally do the simulation. We do it once at the original index position to get the amount of coin transferred originally. Then we do it again at index position 0.

In [15]:
headers = {
    'X-Access-Key': f'{tenderly_access_token}',
    'content-type': 'application/json',
}

#creating an empty DataFrame for the results
columns = ['tx_hash', 'index', 'type', 'raw_amount', 'dollar_value', 'token_contract_address', 'token_name', 'token_dollar_value']
df_results = pd.DataFrame(columns = columns)

# creating a list of tx hashes where the tenderly api returned nothing
tx_hash_problem_list = []

#iterating over every row of the main dataframe (one row is one transaction)
for index, row in df_main.iterrows():
    tx_index_list = [0]
    tx_index_list.append(int(row['transactionIndex'], 0))
    # for each transaction, simulate twice: once for each index
    for tx_index in tx_index_list:
        json_data = {
        'network_id': int(row['chainId'], 0),
        'from': row['from'],
        'to': row['to'],
        'input': row['input'],
        'block_number': row['block_number'],
        'transaction_index': tx_index,
        'simulation_type': 'quick',
        'gas': int(row['gas'], 0),
        'value': int(row['value'], 0),
        'gas_price': int(row['gasPrice'], 0),
        'l1_timestamp': int(row['timeStamp'])
        }
        
        response = requests.post(
        'https://api.tenderly.co/api/v1/account/aurelie2/project/cowswap2/simulate',
        headers=headers,
        json=json_data,
        )

        try:
            for data in response.json()['transaction']['transaction_info']['asset_changes']:
                tx_type = data['type']
                tx_raw_amount = data['raw_amount']
                tx_dollar_value = data['dollar_value']

                #sometimes the following values are empty 
                try:
                    contract_address = data['token_info']['contract_address']
                except:
                    contract_address = 'None'

                try:
                    token_name = data['token_info']['name']
                except:
                    token_name = 'None'
                try:
                    token_dollar_value = data['token_info']['dollar_value']
                except:
                    token_dollar_value = 'None'

                new_row = {
                    'tx_hash' : row['hash'],
                    'index' : tx_index, 
                    'type': tx_type, 
                    'raw_amount': tx_raw_amount, 
                    'dollar_value' : tx_dollar_value, 
                    'token_contract_address': contract_address, 
                    'token_name': token_name, 
                    'token_dollar_value': token_dollar_value
                    }
                df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except:
            tx_hash_problem_list.append(row['hash'])
            


#### creating a new dataframe with only the transactions where tenderly api returned something.
This meanse that there are 2 index values for these good transactions.

In [35]:
grouped = df_results.groupby('tx_hash')
df_results_good = grouped.filter(lambda x: x['index'].nunique() == 2)
df_results_good.head()

,tx_hash,index,type,raw_amount,dollar_value,token_contract_address,token_name,token_dollar_value
0,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,0,Mint,300000000000000000,1050.6360351562500001,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,3502.1201171875
1,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,0,Transfer,300000000000000000,1050.6360351562500001,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,3502.1201171875
2,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,0,Transfer,109634497747398718977119,723.111871526573443924856,0x0026dfbd8dbb6f8d0c88303cc1b1596409fda542,SANSHU!,0.006595660001039505
3,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,0,Transfer,300000000000000000,1051.2659912109375,None,Ethereum,3504.219970703125
4,0xa9a1533c37d53d461be2821ca53bf04a426903809575...,0,Transfer,300000000000000000,1051.2659912109375,None,Ethereum,3504.219970703125


In [36]:
print(
    'number of transactions for which this approach worked:', df_results_good.tx_hash.nunique(),
    "\nnumber of transactions for which this approach did not worked:", len(set(tx_hash_problem_list)),
    '\npercentage of properly simulated transactions:', format(df_results_good.tx_hash.nunique()/(df_results_good.tx_hash.nunique() + len(set(tx_hash_problem_list))), ".2%")
    )


number of transactions for which this approach worked: 120 
number of transactions for which this approach did not worked: 10 
percentage of properly simulated transactions: 92.31%


### Calculate the dollar value of the potential loss for the coins that don't have a dollar value
The logic here is 
1. first calculate the difference in dollar if the simulation gives it to us: dollar_value_diff
2. if we do not have any dollar value for a coin, there will always be a WETH value in the transaction. So we will get the value of the unknown coin based on the WETH value: calulated_dollar_diff
3. we get the loss for that transaction (dollar_diff) by chosing the dollar_value_diff (if exists) or calulated_dollar_diff (if dollar_value_diff does not exist) 
4. we sum the dollar_diff to calculte the total loss over all transaction

#### 1. dollar_value_diff

In [37]:
# when the coin appears multiple times, then the amounts are the same anyway
df_results_good = df_results_good.groupby(['tx_hash', 'index', 'token_contract_address'], as_index=False).first()
df_results_good['dollar_value'] = df_results_good['dollar_value'].astype(float)
df_results_good['dollar_value_diff'] = df_results_good.groupby(['tx_hash', 'token_contract_address'])['dollar_value'].diff()
df_results_good.head()

,tx_hash,index,token_contract_address,type,raw_amount,dollar_value,token_name,token_dollar_value,dollar_value_diff
0,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,2089875520567136769530,1136.330053,0xGasless,0.5437309741973877,NaN
1,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3317909928829407822,11619.719109,WETH,3502.1201171875,NaN
2,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xeaa63125dd63f10874f99cdbbb18410e7fc79dd3,Transfer,294637414594000000000000,4714.316302,Hemule,0.016000399366021156,NaN
3,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,1942114709584941362737,1055.987923,0xGasless,0.5437309741973877,-80.342130
4,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3062478532544235779,10725.167677,WETH,3502.1201171875,-894.551431


#### 2. calculated_dollar_diff

In [38]:
#first if token name is WETH, then the raw_amount need tos be converted. Every transaction has WETH so I will base the analysis on that 
df_results_good['raw_amount'] = df_results_good['raw_amount'].astype(float)
df_results_good.loc[df_results_good['token_name'] == 'WETH', 'raw_amount'] /= 1e+18
df_results_good.head()

,tx_hash,index,token_contract_address,type,raw_amount,dollar_value,token_name,token_dollar_value,dollar_value_diff
0,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,2.089876e+21,1136.330053,0xGasless,0.5437309741973877,NaN
1,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.317910e+00,11619.719109,WETH,3502.1201171875,NaN
2,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xeaa63125dd63f10874f99cdbbb18410e7fc79dd3,Transfer,2.946374e+23,4714.316302,Hemule,0.016000399366021156,NaN
3,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,1.942115e+21,1055.987923,0xGasless,0.5437309741973877,-80.342130
4,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.062479e+00,10725.167677,WETH,3502.1201171875,-894.551431


In [39]:
def calculate_dollar_value(row, df):
    if pd.isna(row['dollar_value']):
        # Filter the DataFrame to get the subset with the same 'tx_hash' and 'index' values
        subset = df[(df['tx_hash'] == row['tx_hash']) & (df['index'] == row['index'])]

        if not subset.empty:
            # Calculate dollar value based on 'WETH' value if the subset is not empty
            weth_row = subset[subset['token_name'] == 'WETH']
            if not weth_row.empty:
                weth_dollar_value = float(weth_row['dollar_value'].values[0])
                unknown_raw_amount = float(row['raw_amount'])
                return weth_dollar_value / unknown_raw_amount

    # If token_name is not NaN or if the subset is empty, return the original dollar_value
    return row['dollar_value']


In [40]:
# Apply the function to each row
# Iterate over each row in the DataFrame
for index, row in df_results_good.iterrows():
    # Call the calculate_dollar_value function and pass the current row and the DataFrame
    calculated_value = calculate_dollar_value(row, df_results_good)
    
    # Update the 'dollar_value' column with the calculated value
    df_results_good.at[index, 'dollar_value'] = calculated_value

df_results_good.head()

,tx_hash,index,token_contract_address,type,raw_amount,dollar_value,token_name,token_dollar_value,dollar_value_diff
0,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,2.089876e+21,1136.330053,0xGasless,0.5437309741973877,NaN
1,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.317910e+00,11619.719109,WETH,3502.1201171875,NaN
2,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xeaa63125dd63f10874f99cdbbb18410e7fc79dd3,Transfer,2.946374e+23,4714.316302,Hemule,0.016000399366021156,NaN
3,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,1.942115e+21,1055.987923,0xGasless,0.5437309741973877,-80.342130
4,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.062479e+00,10725.167677,WETH,3502.1201171875,-894.551431


In [41]:
# we sort to make sure the original index is always on top
df_results_good = df_results_good.sort_values(by = ['tx_hash', 'index'], ascending = [True, False])
df_results_good.head()

,tx_hash,index,token_contract_address,type,raw_amount,dollar_value,token_name,token_dollar_value,dollar_value_diff
3,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,1.942115e+21,1055.987923,0xGasless,0.5437309741973877,-80.342130
4,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.062479e+00,10725.167677,WETH,3502.1201171875,-894.551431
5,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xeaa63125dd63f10874f99cdbbb18410e7fc79dd3,Transfer,2.946374e+23,4714.316302,Hemule,0.016000399366021156,0.000000
0,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,2.089876e+21,1136.330053,0xGasless,0.5437309741973877,NaN
1,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.317910e+00,11619.719109,WETH,3502.1201171875,NaN


In [42]:
# FYI if token_contract_address == 'None', then it's always Ethereum
df_results_good['raw_amount'] = df_results_good['raw_amount'].astype(float)
df_results_good['raw_amount_diff'] = df_results_good.groupby(['tx_hash', 'token_contract_address'])['raw_amount'].transform(lambda x: x[::-1].diff())
df_results_good['calulated_dollar_diff'] = df_results_good['raw_amount_diff'] * df_results_good['dollar_value']
df_results_good.head()

,tx_hash,index,token_contract_address,type,raw_amount,dollar_value,token_name,token_dollar_value,dollar_value_diff,raw_amount_diff,calulated_dollar_diff
3,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,1.942115e+21,1055.987923,0xGasless,0.5437309741973877,-80.342130,-1.477608e+20,-1.560336e+23
4,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.062479e+00,10725.167677,WETH,3502.1201171875,-894.551431,-2.554314e-01,-2.739545e+03
5,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xeaa63125dd63f10874f99cdbbb18410e7fc79dd3,Transfer,2.946374e+23,4714.316302,Hemule,0.016000399366021156,0.000000,0.000000e+00,0.000000e+00
0,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,2.089876e+21,1136.330053,0xGasless,0.5437309741973877,NaN,NaN,NaN
1,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.317910e+00,11619.719109,WETH,3502.1201171875,NaN,NaN,NaN


#### Find the loss for each transaction

In [43]:
df_results_good['dollar_diff'] = np.where(pd.notnull(df_results_good['dollar_value_diff']), df_results_good['dollar_value_diff'], df_results_good['calulated_dollar_diff'])
df_results_good.head()

,tx_hash,index,token_contract_address,type,raw_amount,dollar_value,token_name,token_dollar_value,dollar_value_diff,raw_amount_diff,calulated_dollar_diff,dollar_diff
3,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,1.942115e+21,1055.987923,0xGasless,0.5437309741973877,-80.342130,-1.477608e+20,-1.560336e+23,-80.342130
4,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.062479e+00,10725.167677,WETH,3502.1201171875,-894.551431,-2.554314e-01,-2.739545e+03,-894.551431
5,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,1,0xeaa63125dd63f10874f99cdbbb18410e7fc79dd3,Transfer,2.946374e+23,4714.316302,Hemule,0.016000399366021156,0.000000,0.000000e+00,0.000000e+00,0.000000
0,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0x5fc111f3fa4c6b32eaf65659cfebdeed57234069,Transfer,2.089876e+21,1136.330053,0xGasless,0.5437309741973877,NaN,NaN,NaN,NaN
1,0x02a5aed1bec0904ffe147e0e13cb029d4e4790e42dff...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Transfer,3.317910e+00,11619.719109,WETH,3502.1201171875,NaN,NaN,NaN,NaN


In [44]:
print("this is the total potential loss in dollars for the given transactions", df_results_good.dollar_diff.sum())

this is the total potential loss in dollars for the given transactions -516472.13047353935


### exporting the file

In [45]:
# drop these columns to avoid confusion since the dollar_diff column encompasses these values already
df_results_good.drop(columns=['calulated_dollar_diff', 'dollar_value_diff'], inplace=True)

In [25]:
df_results_good.to_csv('results/results.csv')